In [1]:
#Cargamos las librerías
import numpy as np
import bokeh.plotting as bkp
import bokeh.models as bkm
import bokeh.layouts as bkl
import bokeh.io as bkio
from bokeh.colors import RGB
import pandas as pd
import datetime as dt

#Mostramos las gráficas dentro del notebook
#bkp.output_notebook()

In [2]:
#Cargamos los datos, obtenidos de la página de NOAA, referidas al observatorio de Mauna Loa
#https://www.esrl.noaa.gov/gmd/dv/data

n2odata=pd.read_csv('mlo_N2O_MM.dat', sep="\s+", header=0,comment='#')
co2data=pd.read_csv('co2_mlo_surface-insitu_1_ccgg_MonthlyData.txt', sep="\s+", header=None,comment='#', 
                    names=['site_code','año', 'mes','dia','hora','minuto','segundo','CO2_ppm','incertidumbre', 
                           'num_medidas', 'latitud','longitud','altitud', 'elevacion_nivel_mar', 'elevacion_muestra', 
                           'instrumento', 'flag_control_calidad'])
ch4data=pd.read_csv('ch4_mlo_surface-insitu_1_ccgg_MonthlyData.txt', sep="\s+", header=None,comment='#', 
                    names=['site_code','año', 'mes','dia','hora','minuto','segundo','CH4_ppb','incertidumbre', 
                           'num_medidas', 'latitud','longitud','altitud', 'elevacion_nivel_mar', 'elevacion_muestra',
                           'instrumento', 'flag_control_calidad'])
sf6data=pd.read_csv('mlo_SF6_MM.dat', sep="\s+", header=0,comment='#')

#Si queremos cargar los datos online
#n2odata=pd.read_csv('ftp://aftp.cmdl.noaa.gov/data/hats/n2o/insituGCs/CATS/monthly/mlo_N2O_MM.dat', sep="\s+", 
#                    header=0,comment='#')
#co2data=pd.read_csv('ftp://aftp.cmdl.noaa.gov/data/trace_gases/co2/in-situ/surface/mlo/co2_mlo_surface-insitu_1_ccgg_MonthlyData.txt', sep="\s+",
#                    header=None,comment='#', names=['site_code','año', 'mes','dia','hora','minuto','segundo','CO2_ppm','incertidumbre', 'num_medidas', 
#'latitud','longitud','altitud', 'elevacion_nivel_mar', 'elevacion_muestra', 'instrumento', 'flag_control_calidad'])
#ch4data=pd.read_csv('ftp://aftp.cmdl.noaa.gov/data/trace_gases/ch4/in-situ/surface/mlo/ch4_mlo_surface-insitu_1_ccgg_MonthlyData.txt', sep="\s+",
#                    header=None,comment='#', names=['site_code','año', 'mes','dia','hora','minuto','segundo','CH4_ppb','incertidumbre', 'num_medidas', 
#'latitud','longitud','altitud', 'elevacion_nivel_mar', 'elevacion_muestra', 'instrumento', 'flag_control_calidad'])
#sf6data=pd.read_csv('ftp://aftp.cmdl.noaa.gov/data/hats/sf6/insituGCs/CATS/monthly/mlo_SF6_MM.dat', sep="\s+", 
#                    header=0,comment='#')


In [3]:
#Renombramos las columnas
n2odata.set_axis(['año','mes','N2O_ppb','desv_std','num_muestras'], axis=1, inplace=True)
#co2data.set_axis(['site_code','año', 'mes','dia','hora','minuto','segundo','CO2_ppm','incertidumbre', 
#'num_medidas', 'latitud','longitud','altitud', 'elevacion_nivel_mar', 'elevacion_muestra', 'instrumento', 
#'flag_control_calidad'], axis=1, inplace=True)
#ch4data.set_axis(['site_code','año', 'mes','dia','hora','minuto','segundo','CH4_ppb','incertidumbre', 
#'num_medidas', 'latitud','longitud','altitud', 'elevacion_nivel_mar', 'elevacion_muestra', 'instrumento', 
#'flag_control_calidad'], axis=1, inplace=True)
sf6data.set_axis(['año','mes','SF6_ppt','desv_std','num_muestras'], axis=1, inplace=True)

In [4]:
#En este set de datos, cuando no hay muestras lo indican con -999.99, lo sustituimos por NaN para no representarlo
co2data.replace(to_replace=-999.99,value=float('nan'),inplace=True)
ch4data.replace(to_replace=-999.99,value=float('nan'),inplace=True)

In [5]:
n2odata['Fecha']=pd.to_datetime(n2odata['año'].astype(str)+n2odata['mes'].astype(str),format="%Y%m")
co2data['Fecha']=pd.to_datetime(co2data['año'].astype(str)+co2data['mes'].astype(str),format="%Y%m")
ch4data['Fecha']=pd.to_datetime(ch4data['año'].astype(str)+ch4data['mes'].astype(str),format="%Y%m")
sf6data['Fecha']=pd.to_datetime(sf6data['año'].astype(str)+sf6data['mes'].astype(str),format="%Y%m")

In [6]:
#Leemos las primeras 25 filas para ver que se han cargado bien los datos
n2odata.head(5)

,año,mes,N2O_ppb,desv_std,num_muestras,Fecha
0,1998,1,NaN,NaN,0,1998-01-01
1,1998,2,NaN,NaN,0,1998-02-01
2,1998,3,NaN,NaN,0,1998-03-01
3,1998,4,NaN,NaN,0,1998-04-01
4,1998,5,NaN,NaN,0,1998-05-01


In [7]:
#Cargamos los colores elegidos como paleta para este MOOC
p_azulmarino = RGB(0, 28, 100)
p_azulclaro = RGB(0, 131, 255)
p_amarillo = RGB(244, 255, 84)
p_verde = RGB(172, 255, 165)
p_rojo = RGB(255, 67, 67)

In [8]:
#Represento la cantidad de N2O como una línea verde
p_n2o = bkp.figure(tools=["pan","wheel_zoom","box_zoom","save,reset"], width=600, height=500,
               x_axis_type="datetime", active_drag = "pan", active_scroll = "wheel_zoom")
p_n2o_r=p_n2o.line(source=n2odata,x='Fecha',y='N2O_ppb', color=p_verde, alpha=1,line_width=3, legend="N2O")
p_n2o.add_tools(bkm.HoverTool(renderers=[p_n2o_r], tooltips=[("Media Mensual NO2","@{N2O_ppb}ppb (@mes / @{año})")],mode='vline'))

#Represento la cantidad de CO2 como una línea azul
p_co2 = bkp.figure(tools=["pan","wheel_zoom","box_zoom","save,reset"], width=900, height=500,
               x_axis_type="datetime", active_drag = "pan", active_scroll = "wheel_zoom")
p_co2_r=p_co2.line(source=co2data, x='Fecha',y='CO2_ppm', color=p_azulmarino, alpha=0.7,line_width=3, legend="CO2")
p_co2.add_tools(bkm.HoverTool(renderers=[p_co2_r], tooltips=[("Media Mensual CO2","@{CO2_ppm}ppm  (@mes / @{año})")],mode='vline'))


#Represento la cantidad de CH4 como una línea roja
p_ch4 = bkp.figure(tools=["pan","wheel_zoom","box_zoom","save,reset"], width=900, height=500,
               x_axis_type="datetime", active_drag = "pan", active_scroll = "wheel_zoom")
p_ch4_r=p_ch4.line(source=ch4data, x='Fecha', y='CH4_ppb', color=p_rojo, alpha=1,line_width=3, legend="CH4")
p_ch4.add_tools(bkm.HoverTool(renderers=[p_ch4_r], tooltips=[("Media Mensual CH4","@{CH4_ppb}ppb  (@mes / @{año})")],mode='vline'))

#Represento la cantidad de SF6 como una línea negra
p_sf6 = bkp.figure(tools=["pan","wheel_zoom","box_zoom","save,reset"], width=600, height=500,
               x_axis_type="datetime", active_drag = "pan", active_scroll = "wheel_zoom")
p_sf6_r=p_sf6.line(source=sf6data,x='Fecha',y='SF6_ppt', color='black', alpha=1,line_width=3, legend="SF6")
p_sf6.add_tools(bkm.HoverTool(renderers=[p_sf6_r], tooltips=[("Media Mensual SF6","@{SF6_ppt}ppt (@mes / @{año})")],mode='vline'))

In [9]:
#Indico las propiedades del gráfico
p_n2o.background_fill_color = "white"
p_n2o.background_fill_alpha = 1.

p_n2o.xaxis.axis_label = 'Año'
p_n2o.xaxis.axis_label_text_font = "ADAM"
p_n2o.xaxis.axis_label_text_font_style  = "normal"
p_n2o.xaxis.axis_label_text_font_size = "12pt"
p_n2o.yaxis.axis_label = 'N2O (partes por mil millones)'
p_n2o.yaxis.axis_label_text_font = "ADAM"
p_n2o.yaxis.axis_label_text_font_style  = "normal"
p_n2o.yaxis.axis_label_text_font_size = "12pt"
#p_n2o.y_range = bkm.Range1d(start=280, end=380)

p_n2o.ygrid.grid_line_color=p_azulclaro
p_n2o.ygrid.grid_line_alpha=0.25
p_n2o.xgrid.grid_line_color=p_azulclaro
p_n2o.xgrid.grid_line_alpha=0.0

p_n2o.xaxis.major_tick_line_color = p_rojo
p_n2o.xaxis.major_tick_line_width = 3
p_n2o.xaxis.minor_tick_line_color = p_rojo
p_n2o.xaxis.minor_tick_line_width = 1
p_n2o.yaxis.major_tick_line_color = p_rojo
p_n2o.yaxis.major_tick_line_width = 3
p_n2o.yaxis.minor_tick_line_color = p_rojo
p_n2o.yaxis.minor_tick_line_width = 1

p_n2o.outline_line_width = 7
p_n2o.outline_line_alpha = 0.5
p_n2o.outline_line_color = p_azulclaro

p_n2o.legend.location = "top_left"
p_n2o.legend.click_policy="hide"
p_n2o.legend.label_text_font = "ADAM"


In [10]:
#Indico las propiedades del gráfico


p_co2.background_fill_color = "white"
p_co2.background_fill_alpha = 1.

p_co2.xaxis.axis_label = 'Año'
p_co2.xaxis.axis_label_text_font = "ADAM"
p_co2.xaxis.axis_label_text_font_style  = "normal"
p_co2.xaxis.axis_label_text_font_size = "12pt"
p_co2.yaxis.axis_label = 'CO2 (partes por millón)'
p_co2.yaxis.axis_label_text_font = "ADAM"
p_co2.yaxis.axis_label_text_font_style  = "normal"
p_co2.yaxis.axis_label_text_font_size = "12pt"
p_co2.x_range = bkm.Range1d(start=dt.date(year=1974, month=1, day=1), end=dt.date(year=2017, month=3, day=1))


p_co2.ygrid.grid_line_color=p_azulclaro
p_co2.ygrid.grid_line_alpha=0.25
p_co2.xgrid.grid_line_color=p_azulclaro
p_co2.xgrid.grid_line_alpha=0.0

p_co2.xaxis.major_tick_line_color = p_rojo
p_co2.xaxis.major_tick_line_width = 3
p_co2.xaxis.minor_tick_line_color = p_rojo
p_co2.xaxis.minor_tick_line_width = 1
p_co2.yaxis.major_tick_line_color = p_rojo
p_co2.yaxis.major_tick_line_width = 3
p_co2.yaxis.minor_tick_line_color = p_rojo
p_co2.yaxis.minor_tick_line_width = 1

p_co2.outline_line_width = 7
p_co2.outline_line_alpha = 0.5
p_co2.outline_line_color = p_azulclaro

p_co2.legend.location = "top_left"
p_co2.legend.click_policy="hide"
p_co2.legend.label_text_font = "ADAM"

In [11]:
#Indico las propiedades del gráfico

p_ch4.background_fill_color = "white"
p_ch4.background_fill_alpha = 1.

p_ch4.xaxis.axis_label = 'Año'
p_ch4.xaxis.axis_label_text_font = "ADAM"
p_ch4.xaxis.axis_label_text_font_style  = "normal"
p_ch4.xaxis.axis_label_text_font_size = "12pt"
p_ch4.yaxis.axis_label = 'CH4 (partes por mil millones)'
p_ch4.yaxis.axis_label_text_font = "ADAM"
p_ch4.yaxis.axis_label_text_font_style  = "normal"
p_ch4.yaxis.axis_label_text_font_size = "12pt"
p_ch4.x_range = bkm.Range1d(start=dt.date(year=1974, month=1, day=1), end=dt.date(year=2017, month=3, day=1))

p_ch4.ygrid.grid_line_color=p_azulclaro
p_ch4.ygrid.grid_line_alpha=0.25
p_ch4.xgrid.grid_line_color=p_azulclaro
p_ch4.xgrid.grid_line_alpha=0.0

p_ch4.xaxis.major_tick_line_color = p_rojo
p_ch4.xaxis.major_tick_line_width = 3
p_ch4.xaxis.minor_tick_line_color = p_rojo
p_ch4.xaxis.minor_tick_line_width = 1
p_ch4.yaxis.major_tick_line_color = p_rojo
p_ch4.yaxis.major_tick_line_width = 3
p_ch4.yaxis.minor_tick_line_color = p_rojo
p_ch4.yaxis.minor_tick_line_width = 1

p_ch4.outline_line_width = 7
p_ch4.outline_line_alpha = 0.5
p_ch4.outline_line_color = p_azulclaro

p_ch4.legend.location = "top_left"
p_ch4.legend.click_policy="hide"
p_ch4.legend.label_text_font = "ADAM"

In [12]:
#Indico las propiedades del gráfico

p_sf6.background_fill_color = "white"
p_sf6.background_fill_alpha = 1.

p_sf6.xaxis.axis_label = 'Año'
p_sf6.xaxis.axis_label_text_font = "ADAM"
p_sf6.xaxis.axis_label_text_font_style  = "normal"
p_sf6.xaxis.axis_label_text_font_size = "12pt"
p_sf6.yaxis.axis_label = 'SF6 (partes por billón)'
p_sf6.yaxis.axis_label_text_font = "ADAM"
p_sf6.yaxis.axis_label_text_font_style  = "normal"
p_sf6.yaxis.axis_label_text_font_size = "12pt"
#p_sf6.y_range = bkm.Range1d(start=280, end=380)


p_sf6.ygrid.grid_line_color=p_azulclaro
p_sf6.ygrid.grid_line_alpha=0.25
p_sf6.xgrid.grid_line_color=p_azulclaro
p_sf6.xgrid.grid_line_alpha=0.0

p_sf6.xaxis.major_tick_line_color = p_rojo
p_sf6.xaxis.major_tick_line_width = 3
p_sf6.xaxis.minor_tick_line_color = p_rojo
p_sf6.xaxis.minor_tick_line_width = 1
p_sf6.yaxis.major_tick_line_color = p_rojo
p_sf6.yaxis.major_tick_line_width = 3
p_sf6.yaxis.minor_tick_line_color = p_rojo
p_sf6.yaxis.minor_tick_line_width = 1

p_sf6.outline_line_width = 7
p_sf6.outline_line_alpha = 0.5
p_sf6.outline_line_color = p_azulclaro

p_sf6.legend.location = "top_left"
p_sf6.legend.click_policy="hide"
p_sf6.legend.label_text_font = "ADAM"

In [13]:
bkp.output_file("GasesInvernadero.html", title='Bokeh Plot', mode='cdn', root_dir=None)
grid = bkl.gridplot([[p_co2,p_n2o],[p_ch4,p_sf6]], toolbar_location="above")

#p_co2.output_backend = "svg"
#p_ch4.output_backend = "svg"
#p_n2o.output_backend = "svg"
#p_sf6.output_backend = "svg"

#bkp.show(bkl.column(bkm.Div(text="Media Mensual de Gases de Efecto Invernadero"), grid))
bkp.show(grid)